## Experiments: Sensitive Analysis Hyperparameter RSAST:


It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 30 ,50 and 100 for the selected dataset.

In [274]:
import sys 
import os 
#add sast library path
file_path = os.path.dirname(os.getcwd())+"/sast"

#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.path.dirname(os.getcwd())+"\sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"/sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"\sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

#add cd_diagram library path
file_path = os.path.dirname(os.getcwd())+"\cd_diagram"

#file_path = r"C:\Users\Public\random_sast\cd_diagram"
sys.path.append(file_path)


file_path = os.path.dirname(os.getcwd())+"/cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"\cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"/cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

sys.path


['c:\\Users\\Nicolas R\\random_sast\\ExperimentationRSAST',
 'c:\\ProgramData\\Anaconda3\\python39.zip',
 'c:\\ProgramData\\Anaconda3\\DLLs',
 'c:\\ProgramData\\Anaconda3\\lib',
 'c:\\ProgramData\\Anaconda3',
 '',
 'C:\\Users\\Nicolas R\\AppData\\Roaming\\Python\\Python39\\site-packages',
 'c:\\ProgramData\\Anaconda3\\lib\\site-packages',
 'c:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32',
 'c:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'c:\\ProgramData\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'c:\\ProgramData\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Nicolas R\\.ipython',
 'c:\\Users\\Nicolas R\\random_sast/sast',
 'c:\\Users\\Nicolas R\\random_sast\\sast',
 'c:\\Users\\Nicolas R\\random_sast\\ExperimentationRSAST/sast',
 'c:\\Users\\Nicolas R\\random_sast\\ExperimentationRSAST\\sast',
 'c:\\Users\\Nicolas R\\random_sast\\cd_diagram',
 'c:\\Users\\Nicolas R\\random_sast/cd_diagram',
 'c:\\Users\\Nicolas R\\random_sast\\Experimentation

In [275]:
from sast import *
import pandas as pd
import researchpy
import math
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import seaborn as sns
from cd_function import *
from utils_sast import delete_files_in_directory

### Reading Datasets

In [276]:
# Set directory where the RSAST result csv files are located

directories=[]

default_split_acfandpacf_k_1_10_sast=True
default_split_acfandpacf_k_1_10_notsast=True
default_split_all_k_1_10=True
default_split_all_acfandpacf_k_30_50=False
default_split_maxpacf_none_k_30_50=False

default_split_all_acfandpacf_maxpacf_none_maxacf_k_1_50_p_1_100=False

resampling_acfandpacf_k_1_10_sast=False

if default_split_acfandpacf_k_1_10_sast:
    directories.append(os.getcwd()+'/results_rsast/Server17_Comparison_RSAST')
if default_split_acfandpacf_k_1_10_notsast:
    directories.append(os.getcwd()+'/results_rsast/Server17_not_used_sast')
if default_split_all_k_1_10:
    directories.append(os.getcwd()+'/results_rsast/Server17_Comparison_RSAST_All')   
if default_split_all_acfandpacf_k_30_50:
    directories.append(os.getcwd()+'/results_rsast/Server17_Comparison_RSAST_Extra_1')    
if default_split_maxpacf_none_k_30_50:
    directories.append(os.getcwd()+'/results_rsast/Server17_Comparison_RSAST_Extra_2')    
if resampling_acfandpacf_k_1_10_sast:
    directories.append(os.getcwd()+'/results_rsast/Server17_Resampling_Comparison_RSAST')     
if default_split_all_acfandpacf_maxpacf_none_maxacf_k_1_50_p_1_100:
    directories.append(os.getcwd()+'/results_rsast/Server16_Hyperparameter_Tunning')
    directories.append(os.getcwd()+'/results_rsast/Server17_Hyperparameter_Tunning')


# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for directory in directories:
    for filename in os.listdir(directory):
        # Check if the file starts with "df_overall_tunning" and ends with ".csv"
        if filename.startswith("df_all_overall_tunning") and filename.endswith(".csv"):
            # Read the csv file into a dataframe
            filepath = os.path.join(directory, filename)
            df = pd.read_csv(filepath)
            df['rpoint']=df['rpoint'].astype(str)
            df['nb_per_class']=df['nb_per_class'].astype(str)
            df['rpoint']=df['rpoint'].replace("(lenthg ts)//2","half_len")
            df['nb_per_class']=df['nb_per_class'].replace("(max instances per class)//2","half_instance")
            df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
            df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
            df['directory']=directory
            # Append the dataframe to the list
            dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)
# df_result.head(3)


C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_3828\967067762.py:47: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_3828\967067762.py:48: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_3828\967067762.py:47: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_3828\967067762.py:48: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace(

In [277]:
print("Total DS:" + str(len(df_result['dataset_name'].unique())))

Total DS:128


In [278]:
pv_all=pd.pivot_table(df_result,index='dataset_name',values='accuracy',aggfunc="count")
ds_complete=pv_all[pv_all.accuracy==max(pv_all.accuracy)].reset_index().dataset_name.unique()
ds_complete
df_result=df_result[df_result.dataset_name.isin(ds_complete)]

In [279]:
print("Complete (max runs) per DS:" + str(len(df_result['dataset_name'].unique())))

Complete (max runs) per DS:72


In [280]:
pv_all=pd.pivot_table(df_result,index='len_method',values='accuracy',aggfunc="count")
len_complete=pv_all[pv_all.accuracy==max(pv_all.accuracy)].reset_index().len_method.unique()
len_complete
df_result=df_result[df_result.len_method.isin(len_complete)]

In [281]:
print("Complete (max runs) per Len Method:" + str(len(df_result['dataset_name'].unique())))

Complete (max runs) per Len Method:72


In [282]:
#get ds tested overall
print("Used Datasets:")
pd.pivot_table(df_result, index="dataset_name", values="accuracy", aggfunc="count").reset_index()["dataset_name"].unique()

Used Datasets:


array(['Adiac', 'ArrowHead', 'BME', 'Beef', 'BeetleFly', 'BirdChicken',
       'CBF', 'Car', 'Chinatown', 'ChlorineConcentration', 'Coffee',
       'CricketX', 'CricketY', 'CricketZ', 'Crop', 'DiatomSizeReduction',
       'DistalPhalanxOutlineAgeGroup', 'DistalPhalanxOutlineCorrect',
       'DistalPhalanxTW', 'DodgerLoopDay', 'DodgerLoopGame',
       'DodgerLoopWeekend', 'ECG200', 'ECG5000', 'ECGFiveDays',
       'Earthquakes', 'ElectricDevices', 'FaceAll', 'FaceFour',
       'FacesUCR', 'FiftyWords', 'FreezerRegularTrain',
       'FreezerSmallTrain', 'Fungi', 'GunPoint', 'GunPointAgeSpan',
       'GunPointMaleVersusFemale', 'GunPointOldVersusYoung', 'Ham',
       'Herring', 'InsectWingbeatSound', 'ItalyPowerDemand', 'Meat',
       'MedicalImages', 'MelbournePedestrian',
       'MiddlePhalanxOutlineAgeGroup', 'MiddlePhalanxOutlineCorrect',
       'MiddlePhalanxTW', 'MoteStrain', 'PhalangesOutlinesCorrect',
       'Plane', 'PowerCons', 'ProximalPhalanxOutlineAgeGroup',
       'ProximalP

In [283]:
#get ds tested overall
print("Total Param Combination:" + str(len(df_result['classifier_name'].unique())))

Total Param Combination:16


In [284]:
#get len methods of generated datasets
df_result.classifier_name.unique()

array(['ACF&PACF: n_random_points=10 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=10 nb_inst_per_class=10',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=10',
       'Max PACF: n_random_points=10 nb_inst_per_class=1',
       'Max PACF: n_random_points=10 nb_inst_per_class=10',
       'Max PACF: n_random_points=30 nb_inst_per_class=1',
       'Max PACF: n_random_points=30 nb_inst_per_class=10',
       'None: n_random_points=10 nb_inst_per_class=1',
       'None: n_random_points=10 nb_inst_per_class=10',
       'None: n_random_points=30 nb_inst_per_class=1',
       'None: n_random_points=30 nb_inst_per_class=10',
       'all: n_random_points=10 nb_inst_per_class=1',
       'all: n_random_points=10 nb_inst_per_class=10',
       'all: n_random_points=30 nb_inst_per_class=1',
       'all: n_random_points=30 nb_inst_per_class=10'], dtype=object)

In [285]:
#get columns of generated datasets
df_result.columns.unique()

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method', 'directory'],
      dtype='object')

In [286]:
df_used_sast=pd.read_excel("DataSetsUCLASummary.xlsx", dtype=str)
df_used_sast.columns
ds_bake=df_used_sast[df_used_sast['BAKE OFF']=='Y'].Name.unique()
len(ds_bake)


85

In [287]:

df_print=pd.pivot_table(df_result, index="dataset_name", values="accuracy", columns="classifier_name").reset_index().round(2)

if not resampling_acfandpacf_k_1_10_sast:
       df_print.to_csv("results_default_split.csv")
else: 
       df_print.to_csv("results_10resampling.csv")
df_print


classifier_name,dataset_name,ACF&PACF: n_random_points=10 nb_inst_per_class=1,ACF&PACF: n_random_points=10 nb_inst_per_class=10,ACF&PACF: n_random_points=30 nb_inst_per_class=1,ACF&PACF: n_random_points=30 nb_inst_per_class=10,Max PACF: n_random_points=10 nb_inst_per_class=1,Max PACF: n_random_points=10 nb_inst_per_class=10,Max PACF: n_random_points=30 nb_inst_per_class=1,Max PACF: n_random_points=30 nb_inst_per_class=10,None: n_random_points=10 nb_inst_per_class=1,None: n_random_points=10 nb_inst_per_class=10,None: n_random_points=30 nb_inst_per_class=1,None: n_random_points=30 nb_inst_per_class=10,all: n_random_points=10 nb_inst_per_class=1,all: n_random_points=10 nb_inst_per_class=10,all: n_random_points=30 nb_inst_per_class=1,all: n_random_points=30 nb_inst_per_class=10
0,Adiac,0.63,0.58,0.61,0.57,0.70,0.50,0.45,0.50,0.71,0.62,0.59,0.64,0.60,0.55,0.56,0.55
1,ArrowHead,0.75,0.79,0.75,0.80,0.76,0.69,0.67,0.71,0.70,0.72,0.63,0.71,0.73,0.74,0.72,0.75
2,BME,0.83,0.83,0.83,0.83,0.77,0.80,0.74,0.81,0.76,0.87,0.77,0.86,0.85,0.87,0.88,0.87
3,Beef,0.67,0.68,0.66,0.69,0.51,0.64,0.51,0.65,0.56,0.70,0.58,0.73,0.79,0.83,0.75,0.83
4,BeetleFly,0.78,0.77,0.80,0.78,0.64,0.79,0.62,0.79,0.54,0.77,0.68,0.73,0.77,0.85,0.74,0.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,TwoPatterns,0.97,1.00,0.95,1.00,0.75,0.96,0.81,0.85,0.63,0.97,0.81,0.89,0.98,1.00,0.98,1.00
68,UMD,0.96,0.97,0.93,0.98,0.67,0.83,0.70,0.85,0.76,0.94,0.74,0.95,0.96,0.98,0.96,0.97
69,Wafer,1.00,1.00,1.00,1.00,0.98,1.00,0.99,1.00,0.98,1.00,0.99,1.00,1.00,1.00,1.00,1.00
70,Wine,0.85,0.78,0.78,0.78,0.69,0.79,0.70,0.81,0.61,0.83,0.61,0.84,0.82,0.88,0.84,0.89


### Hyperparameter Tunning RSAST 

#### Accuracy: Subsequence Lenght Method 

In [288]:
# Summary statistics for a Series (single variable)
summ_ds=df_result
summ_ds=researchpy.summary_cont(summ_ds.groupby(['dataset_name'])['accuracy'], conf = 0.95)
summ_ds

,N,Mean,SD,SE,95% Conf.,Interval
dataset_name,,,,,,
Adiac,80,0.5839,0.0683,0.0076,0.5687,0.5991
ArrowHead,80,0.7253,0.0497,0.0056,0.7142,0.7364
BME,80,0.8226,0.0545,0.0061,0.8104,0.8347
Beef,80,0.6742,0.1106,0.0124,0.6496,0.6988
BeetleFly,80,0.7438,0.1101,0.0123,0.7193,0.7682
...,...,...,...,...,...,...
TwoPatterns,80,0.9085,0.1133,0.0127,0.8833,0.9337
UMD,80,0.8844,0.1140,0.0128,0.8590,0.9098
Wafer,80,0.9939,0.0076,0.0008,0.9923,0.9956


In [289]:
stats = df_result.groupby(['len_method'])['accuracy'].agg(['mean', 'count', 'std'])

ci95_hi = []
ci95_lo = []

for i in stats.index:
    m, c, s = stats.loc[i]
    ci95_hi.append(m + 1.96*s/math.sqrt(c))
    ci95_lo.append(m - 1.96*s/math.sqrt(c))

stats['ci95_hi'] = ci95_hi
stats['ci95_lo'] = ci95_lo
print(stats.head(10))

                mean  count       std   ci95_hi   ci95_lo
len_method                                               
ACF&PACF    0.827158   1440  0.137813  0.834276  0.820040
Max PACF    0.786544   1440  0.150697  0.794328  0.778761
None        0.785727   1440  0.146854  0.793313  0.778142
all         0.831262   1440  0.140306  0.838509  0.824015


#### Generate Boxplot Tunning Hyperparameter

In [290]:
df_result.nb_per_class.unique()

array(['1', '10'], dtype=object)

In [291]:
df_result.len_method.unique()

array(['ACF&PACF', 'Max PACF', 'None', 'all'], dtype=object)

In [292]:
#generate hyperparameter tuning boxplots
delete_files_in_directory("images_boxplot_acc/")
for k, ints in enumerate(df_result.nb_per_class.unique()):
    for len_m in df_result.len_method.unique():
        df_to_cd=df_result[df_result.classifier_name.str.contains(r'^'+len_m+':.*nb_inst_per_class='+str(ints)+'$')]
        # Plot
        fig, ax = plt.subplots()

        sns.boxplot(data=df_to_cd, x='rpoint', y='accuracy', palette="Blues")
        
        max_bx=max(df_to_cd.accuracy)
        min_bx=min(df_to_cd.accuracy)
        
        ax.set_ylim(min_bx,max_bx)
        # Axis details
        ax.set(xlabel='number of random points', ylabel='accuracy', title='Accuracy Boxplot for Len method:'+len_m+' and N° of Instances:'+ints)
        #plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )

        # save plot
        plt.savefig('images_boxplot_acc/boxplot_acc'+len_m+ints+'.png')


All files deleted successfully.


In [293]:
df_result.classifier_name.unique()

array(['ACF&PACF: n_random_points=10 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=10 nb_inst_per_class=10',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=10',
       'Max PACF: n_random_points=10 nb_inst_per_class=1',
       'Max PACF: n_random_points=10 nb_inst_per_class=10',
       'Max PACF: n_random_points=30 nb_inst_per_class=1',
       'Max PACF: n_random_points=30 nb_inst_per_class=10',
       'None: n_random_points=10 nb_inst_per_class=1',
       'None: n_random_points=10 nb_inst_per_class=10',
       'None: n_random_points=30 nb_inst_per_class=1',
       'None: n_random_points=30 nb_inst_per_class=10',
       'all: n_random_points=10 nb_inst_per_class=1',
       'all: n_random_points=10 nb_inst_per_class=10',
       'all: n_random_points=30 nb_inst_per_class=1',
       'all: n_random_points=30 nb_inst_per_class=10'], dtype=object)

In [294]:
delete_files_in_directory("images_boxplot_acc_per_ds/")
#generate hyperparameter tuning boxplots
for ds in df_result.dataset_name.unique():
    for k, ints in enumerate(df_result.nb_per_class.unique()):
        for len_m in df_result.len_method.unique():
            df_to_cd=df_result[df_result.classifier_name.str.contains(r'^'+len_m+':')]
            df_to_cd=df_to_cd[df_to_cd.dataset_name==ds]
            df_to_cd["time"]=df_to_cd['cweights_time']+df_to_cd['fsubsequence_time']+df_to_cd['tdataset_time']
            df_to_cd["time"]=df_to_cd["time"]/60
            max_bx=max(df_to_cd.accuracy)
            min_bx=min(df_to_cd.accuracy)
            df_to_cd=df_to_cd[df_to_cd.classifier_name.str.contains(r':.*nb_inst_per_class='+str(ints)+'$')]    
            
            # Plot
            fig, ax = plt.subplots()
            sns.boxplot(data=df_to_cd, x='rpoint', y='accuracy', palette="Blues")
            ax.set_ylim(min_bx,max_bx)

            # Axis details
            ax.set(xlabel='number of random points', ylabel='accuracy', title='Accuracy Boxplot for Len method:'+len_m+' and N° of Instances:'+ints)
            #plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )

            # save plot
            plt.savefig('images_boxplot_acc_per_ds/boxplot_acc'+len_m+ints+'_'+ds+'.png')

All files deleted successfully.


C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_3828\1503050523.py:17: UserWarning: Attempting to set identical bottom == top == 1.0 results in singular transformations; automatically expanding.
  ax.set_ylim(min_bx,max_bx)
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_3828\1503050523.py:17: UserWarning: Attempting to set identical bottom == top == 1.0 results in singular transformations; automatically expanding.
  ax.set_ylim(min_bx,max_bx)
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_3828\1503050523.py:17: UserWarning: Attempting to set identical bottom == top == 0.8817204301075269 results in singular transformations; automatically expanding.
  ax.set_ylim(min_bx,max_bx)
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_3828\1503050523.py:17: UserWarning: Attempting to set identical bottom == top == 0.8817204301075269 results in singular transformations; automatically expanding.
  ax.set_ylim(min_bx,max_bx)
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_3828\1503050523.py:17: Use

In [295]:
delete_files_in_directory("images_boxplot_time/")
#generate hyperparameter tuning boxplots
for k, ints in enumerate(df_result.nb_per_class.unique()):
    for len_m in df_result.len_method.unique():
                
        df_to_cd=df_result[df_result.classifier_name.str.contains(r'^'+len_m+':.*nb_inst_per_class='+str(ints)+'$')]
        df_to_cd["time"]=df_to_cd['cweights_time']+df_to_cd['fsubsequence_time']+df_to_cd['tdataset_time']
        df_to_cd["time"]=df_to_cd["time"]/60

        # Plot
        fig, ax = plt.subplots()
        sns.boxplot(data=df_to_cd, x='rpoint', y='time', palette="Blues")      
        max_bx=max(df_to_cd.time)
        min_bx=min(df_to_cd.time)
        ax.set_ylim(min_bx,max_bx)
        # Axis details
        ax.set(xlabel='number of random points', ylabel='time', title='Time Boxplot for Len method:'+len_m+' and N° of Instances:'+ints)
        #plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )

        # save plot
        plt.savefig('images_boxplot_time/boxplot_time'+len_m+ints+'.png')


All files deleted successfully.


C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_3828\191012824.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_cd["time"]=df_to_cd['cweights_time']+df_to_cd['fsubsequence_time']+df_to_cd['tdataset_time']
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_3828\191012824.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_cd["time"]=df_to_cd["time"]/60
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_3828\191012824.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [296]:
delete_files_in_directory("images_boxplot_time_per_ds/")
#generate hyperparameter tuning boxplots
for ds in df_result.dataset_name.unique():
    for k, ints in enumerate(df_result.nb_per_class.unique()):
        for len_m in df_result.len_method.unique():      

            df_to_cd=df_result[df_result.classifier_name.str.contains(r'^'+len_m+':')]
            df_to_cd=df_to_cd[df_to_cd.dataset_name==ds]

            df_to_cd["time"]=df_to_cd['cweights_time']+df_to_cd['fsubsequence_time']+df_to_cd['tdataset_time']
            df_to_cd["time"]=df_to_cd["time"]/60
            max_bx=max(df_to_cd.time)
            min_bx=min(df_to_cd.time)

            df_to_cd=df_to_cd[df_to_cd.classifier_name.str.contains(r':.*nb_inst_per_class='+str(ints)+'$')]

            # Plot
            fig, ax = plt.subplots()
                       
            #order=list(df_to_cd)
            sns.boxplot(data=df_to_cd, x='rpoint', y='time', palette="Blues")

            ax.set_ylim(min_bx,max_bx)
            # Axis details
            ax.set(xlabel='number of random points', ylabel='time', title='Time Boxplot for Len method:'+len_m+' and N° of Instances:'+ints)
            #plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )

            # save plot
            plt.savefig('images_boxplot_time_per_ds/boxplot_time'+len_m+ints+'_'+ds+'.png')


All files deleted successfully.


#### Generate CD Diagram Tunning Hyperparameter

In [297]:
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name','len_method'], aggfunc='mean')
df_perf=df_perf.reset_index()
df_perf.classifier_name.unique()

array(['ACF&PACF: n_random_points=10 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=10 nb_inst_per_class=10',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=10',
       'Max PACF: n_random_points=10 nb_inst_per_class=1',
       'Max PACF: n_random_points=10 nb_inst_per_class=10',
       'Max PACF: n_random_points=30 nb_inst_per_class=1',
       'Max PACF: n_random_points=30 nb_inst_per_class=10',
       'None: n_random_points=10 nb_inst_per_class=1',
       'None: n_random_points=10 nb_inst_per_class=10',
       'None: n_random_points=30 nb_inst_per_class=1',
       'None: n_random_points=30 nb_inst_per_class=10',
       'all: n_random_points=10 nb_inst_per_class=1',
       'all: n_random_points=10 nb_inst_per_class=10',
       'all: n_random_points=30 nb_inst_per_class=1',
       'all: n_random_points=30 nb_inst_per_class=10'], dtype=object)

In [298]:
delete_files_in_directory("images_cd_diagram/")
#generate hyperparameter tuning cd diagrams

best_comb_by_method=[]
best=""
for len_m in df_perf.len_method.unique():
   df_to_cd=df_perf[df_perf.classifier_name.str.contains(r'^'+len_m+':.*$')]
   draw_cd_diagram(df_to_cd, labels=True, title=len_m +" comparison", fname='images_cd_diagram/cd-diagram_'+len_m+'.png')
   _, average_ranks, _ = wilcoxon_holm(df_perf=df_to_cd)
   min_rank= min(average_ranks)
   average_ranks=pd.DataFrame(average_ranks)
   best=average_ranks[average_ranks[0]==min_rank][0].index
   best_comb_by_method.append(best[0])



All files deleted successfully.
{0: 'ACF&PACF: n_random_points=10 nb_inst_per_class=1', 1: 'ACF&PACF: n_random_points=30 nb_inst_per_class=1', 2: 'ACF&PACF: n_random_points=10 nb_inst_per_class=10', 3: 'ACF&PACF: n_random_points=30 nb_inst_per_class=10'}
[2, 3]
{0: 'Max PACF: n_random_points=10 nb_inst_per_class=1', 1: 'Max PACF: n_random_points=30 nb_inst_per_class=1', 2: 'Max PACF: n_random_points=10 nb_inst_per_class=10', 3: 'Max PACF: n_random_points=30 nb_inst_per_class=10'}
{0: 'None: n_random_points=10 nb_inst_per_class=1', 1: 'None: n_random_points=30 nb_inst_per_class=1', 2: 'None: n_random_points=10 nb_inst_per_class=10', 3: 'None: n_random_points=30 nb_inst_per_class=10'}
[0, 1]
{0: 'all: n_random_points=10 nb_inst_per_class=1', 1: 'all: n_random_points=30 nb_inst_per_class=1', 2: 'all: n_random_points=10 nb_inst_per_class=10', 3: 'all: n_random_points=30 nb_inst_per_class=10'}
[0, 1]
[2, 3]


In [299]:
best_comb_by_method

['ACF&PACF: n_random_points=30 nb_inst_per_class=10',
 'Max PACF: n_random_points=30 nb_inst_per_class=10',
 'None: n_random_points=30 nb_inst_per_class=10',
 'all: n_random_points=10 nb_inst_per_class=10']

In [300]:
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name','len_method'], aggfunc='mean')
df_perf=df_perf.reset_index()
df_to_cd=df_perf[df_perf.classifier_name.isin(best_comb_by_method)]

In [301]:
draw_cd_diagram(df_to_cd, labels=True, title="Best combination comparison", fname='images_cd_diagram/cd-diagram_best_com.png')


{0: 'Max PACF: n_random_points=30 nb_inst_per_class=10', 1: 'None: n_random_points=30 nb_inst_per_class=10', 2: 'ACF&PACF: n_random_points=30 nb_inst_per_class=10', 3: 'all: n_random_points=10 nb_inst_per_class=10'}
[0, 1]
[2, 3]


In [302]:
df_perf_all=df_perf[['dataset_name','len_method','accuracy']]
df_perf_all.rename(columns = {'len_method':'classifier_name'}, inplace = True)
df_perf_all=pd.pivot_table(df_perf_all, index=["dataset_name","classifier_name"]).reset_index()

C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_3828\293824979.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_perf_all.rename(columns = {'len_method':'classifier_name'}, inplace = True)


In [303]:
draw_cd_diagram(df_perf_all, labels=True, fname='images_cd_diagram/cd-diagram_all_methods.png')

{0: 'None', 1: 'Max PACF', 2: 'ACF&PACF', 3: 'all'}
[0, 1]
[2, 3]


#### Focus on most accurate lenght method

##### Overall Accuracy

In [304]:
# highest accurate hyperparameters
best_comb_by_method

['ACF&PACF: n_random_points=30 nb_inst_per_class=10',
 'Max PACF: n_random_points=30 nb_inst_per_class=10',
 'None: n_random_points=30 nb_inst_per_class=10',
 'all: n_random_points=10 nb_inst_per_class=10']

In [305]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result
#df_result_acc=df_result_acc[df_result_acc["dataset_name"]=="Fungi"]

pivot = pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')
pivot = pivot.reindex(columns=["half_instance","1","10","30","50","100","1000","10000"], index=["half_len","1","10","30","50","100","1000","10000"])
pivot

nb_per_class,half_instance,1,10,30,50,100,1000,10000
rpoint,,,,,,,,
half_len,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,0.788595,0.824341,NaN,NaN,NaN,NaN,NaN
30,NaN,0.790650,0.827105,NaN,NaN,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [306]:
# create a pivot table with the variatioon of score by hyperparameter
pivot = pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')
pivot = pivot.reindex(columns=["half_instance","1","10","30","50","100","1000","10000"], index=["half_len","1","10","30","50","100","1000","10000"])
pivot

nb_per_class,half_instance,1,10,30,50,100,1000,10000
rpoint,,,,,,,,
half_len,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,0.020597,0.019937,NaN,NaN,NaN,NaN,NaN
30,NaN,0.023089,0.019885,NaN,NaN,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Overall time complexity

In [307]:
df_result_acc.time.sum()/(60*60*24)

12.258387559274281

In [308]:
# create a pivot table with the mean of time spent in minutes
pivot=pd.pivot_table(df_result_acc, values=['time','cweights_time','fsubsequence_time','tdataset_time','tclassifier_time'], index=['dataset_name'], aggfunc='sum')/60
pivot = pivot.reindex(columns=['cweights_time','fsubsequence_time','tdataset_time','tclassifier_time','time'])
pivot

,cweights_time,fsubsequence_time,tdataset_time,tclassifier_time,time
dataset_name,,,,,
Adiac,0.463301,8.145984,877.130039,7.891883,893.716162
ArrowHead,0.093649,2.876331,14.803851,0.155317,17.939097
BME,0.051486,0.459542,3.173957,0.043407,3.733848
Beef,0.200932,37.720246,60.032112,0.047557,98.026007
BeetleFly,0.138300,33.720243,16.783420,0.021030,50.677934
...,...,...,...,...,...
TwoPatterns,0.119636,0.617728,119.875390,5.683790,126.311355
UMD,0.059536,0.633915,5.205034,0.128314,6.034467
Wafer,0.106447,0.476467,51.550980,1.757995,53.900484


In [309]:
# create a pivot table with the mean of time spent in minutes by random points and instances per class
pivot = pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')/60
pivot = pivot.reindex(columns=["half_instance","1","10","30","50","100","1000","10000"], index=["half_len","1","10","30","50","100","1000","10000"])
pivot

nb_per_class,half_instance,1,10,30,50,100,1000,10000
rpoint,,,,,,,,
half_len,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,0.368510,2.938477,NaN,NaN,NaN,NaN,NaN
30,NaN,0.928246,8.023154,NaN,NaN,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Calculate weights time complexity

In [310]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10
rpoint,,
10,0.104732,0.103949
30,0.103705,0.104504


##### Finding subsequences time complexity

In [311]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10
rpoint,,
10,1.333338,11.307434
30,1.424809,11.551143


##### Transform Dataset time complexity

In [312]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10
rpoint,,
10,17.689851,160.498545
30,51.122587,464.089302


##### Classifier time complexity

In [313]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10
rpoint,,
10,2.967764,4.387551
30,3.033969,5.621210
